<a href="https://colab.research.google.com/github/Remonah-3/Github_Assignment/blob/master/Deep_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class FC:
    def __init__(self, n_in, n_out, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_in, n_out)
        self.B = initializer.B(n_out)

    def forward(self, X):
        self.X = X
        A = np.dot(X, self.W) + self.B
        return A

    def backward(self, dA):
        dW = np.dot(self.X.T, dA)
        dB = np.sum(dA, axis=0)
        dZ = np.dot(dA, self.W.T)
        # Update weights
        self.W, self.B = self.optimizer.update(self.W, self.B, dW, dB)
        return dZ


In [2]:
class SimpleInitializer:

    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, n_in, n_out):
        return np.random.randn(n_in, n_out) * self.sigma

    def B(self, n_out):
        return np.zeros(n_out)


In [3]:
class SGD:

    def __init__(self, lr):
        self.lr = lr

    def update(self, W, B, dW, dB):
        W_new = W - self.lr * dW
        B_new = B - self.lr * dB
        return W_new, B_new


In [4]:
class Tanh:
    def forward(self, X):
        self.Z = np.tanh(X)
        return self.Z

    def backward(self, dA):
        return dA * (1 - self.Z**2)

class Softmax:
    def forward(self, X):
        exps = np.exp(X - np.max(X, axis=1, keepdims=True))
        self.Z = exps / np.sum(exps, axis=1, keepdims=True)
        return self.Z

    def backward(self, Y):
        # Cross-entropy loss derivative
        batch_size = Y.shape[0]
        return (self.Z - Y) / batch_size


In [5]:
class ReLU:
    def forward(self, X):
        self.X = X
        return np.maximum(0, X)

    def backward(self, dA):
        dZ = dA.copy()
        dZ[self.X <= 0] = 0
        return dZ


In [6]:
class XavierInitializer:
    def W(self, n_in, n_out):
        sigma = 1 / np.sqrt(n_in)
        return np.random.randn(n_in, n_out) * sigma
    def B(self, n_out):
        return np.zeros(n_out)

class HeInitializer:
    def W(self, n_in, n_out):
        sigma = np.sqrt(2 / n_in)
        return np.random.randn(n_in, n_out) * sigma
    def B(self, n_out):
        return np.zeros(n_out)


In [7]:
class AdaGrad:
    def __init__(self, lr, epsilon=1e-7):
        self.lr = lr
        self.epsilon = epsilon
        self.H_W = 0
        self.H_B = 0

    def update(self, W, B, dW, dB):
        self.H_W += dW**2
        self.H_B += dB**2
        W_new = W - self.lr * dW / (np.sqrt(self.H_W) + self.epsilon)
        B_new = B - self.lr * dB / (np.sqrt(self.H_B) + self.epsilon)
        return W_new, B_new


In [8]:
class ScratchDeepNeuralNetworkClassifier:
    def __init__(self, layers):
        self.layers = layers  # list of (FC, activation) tuples

    def forward(self, X):
        for fc, act in self.layers:
            X = fc.forward(X)
            X = act.forward(X)
        return X

    def backward(self, Y):
        dA = None
        for fc, act in reversed(self.layers):
            if isinstance(act, Softmax):
                dA = act.backward(Y)
            else:
                dA = act.backward(dA)
            dA = fc.backward(dA)

    def fit(self, X, Y, epochs=10):
        for _ in range(epochs):
            self.forward(X)
            self.backward(Y)

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


In [9]:
# Dummy dataset
X = np.random.randn(100, 784)
Y = np.zeros((100, 10))
Y[np.arange(100), np.random.randint(0,10,100)] = 1

# Build network
lr = 0.01
sigma = 0.1
optimizer = SGD(lr)
layers = [
    (FC(784, 128, SimpleInitializer(sigma), optimizer), Tanh()),
    (FC(128, 64, SimpleInitializer(sigma), optimizer), ReLU()),
    (FC(64, 10, SimpleInitializer(sigma), optimizer), Softmax())
]

model = ScratchDeepNeuralNetworkClassifier(layers)
model.fit(X, Y, epochs=5)
preds = model.predict(X)
accuracy = np.mean(preds == np.argmax(Y, axis=1))
print("Accuracy:", accuracy)


Accuracy: 0.11


In [10]:
import numpy as np

class FC:
    def __init__(self, n_in, n_out, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_in, n_out)
        self.B = initializer.B(n_out)

    def forward(self, X):
        self.X = X
        return np.dot(X, self.W) + self.B

    def backward(self, dA):
        dW = np.dot(self.X.T, dA)
        dB = np.sum(dA, axis=0)
        dZ = np.dot(dA, self.W.T)
        self.W, self.B = self.optimizer.update(self.W, self.B, dW, dB)
        return dZ

# Initializers

class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_in, n_out):
        return np.random.randn(n_in, n_out) * self.sigma
    def B(self, n_out):
        return np.zeros(n_out)

class XavierInitializer:
    def W(self, n_in, n_out):
        sigma = 1 / np.sqrt(n_in)
        return np.random.randn(n_in, n_out) * sigma
    def B(self, n_out):
        return np.zeros(n_out)

class HeInitializer:
    def W(self, n_in, n_out):
        sigma = np.sqrt(2 / n_in)
        return np.random.randn(n_in, n_out) * sigma
    def B(self, n_out):
        return np.zeros(n_out)

# Optimizers

class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, W, B, dW, dB):
        return W - self.lr * dW, B - self.lr * dB

class AdaGrad:
    def __init__(self, lr, epsilon=1e-7):
        self.lr = lr
        self.epsilon = epsilon
        self.H_W = 0
        self.H_B = 0
    def update(self, W, B, dW, dB):
        self.H_W += dW**2
        self.H_B += dB**2
        W_new = W - self.lr * dW / (np.sqrt(self.H_W) + self.epsilon)
        B_new = B - self.lr * dB / (np.sqrt(self.H_B) + self.epsilon)
        return W_new, B_new

# Activation Functions

class Tanh:
    def forward(self, X):
        self.Z = np.tanh(X)
        return self.Z
    def backward(self, dA):
        return dA * (1 - self.Z**2)

class Softmax:
    def forward(self, X):
        exps = np.exp(X - np.max(X, axis=1, keepdims=True))
        self.Z = exps / np.sum(exps, axis=1, keepdims=True)
        return self.Z
    def backward(self, Y):
        return (self.Z - Y) / Y.shape[0]

class ReLU:
    def forward(self, X):
        self.X = X
        return np.maximum(0, X)
    def backward(self, dA):
        dZ = dA.copy()
        dZ[self.X <= 0] = 0
        return dZ

# Deep Neural Network Class

class ScratchDeepNeuralNetworkClassifier:
    def __init__(self, layers):
        self.layers = layers  # list of (FC, activation) tuples

    def forward(self, X):
        for fc, act in self.layers:
            X = fc.forward(X)
            X = act.forward(X)
        return X

    def backward(self, Y):
        dA = None
        for fc, act in reversed(self.layers):
            if isinstance(act, Softmax):
                dA = act.backward(Y)
            else:
                dA = act.backward(dA)
            dA = fc.backward(dA)

    def fit(self, X, Y, epochs=10):
        for _ in range(epochs):
            self.forward(X)
            self.backward(Y)

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

# Training

X = np.random.randn(100, 784)
Y = np.zeros((100, 10))
Y[np.arange(100), np.random.randint(0, 10, 100)] = 1

# Build network
lr = 0.01
sigma = 0.1
optimizer = SGD(lr)
layers = [
    (FC(784, 128, SimpleInitializer(sigma), optimizer), Tanh()),
    (FC(128, 64, SimpleInitializer(sigma), optimizer), ReLU()),
    (FC(64, 10, SimpleInitializer(sigma), optimizer), Softmax())
]

model = ScratchDeepNeuralNetworkClassifier(layers)
model.fit(X, Y, epochs=5)
preds = model.predict(X)
accuracy = np.mean(preds == np.argmax(Y, axis=1))
print("Accuracy:", accuracy)


Accuracy: 0.13
